# 0. 라이브러리

In [9]:
import pandas as pd
import numpy as np
import time

import OpenDartReader

In [10]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [11]:
api_key = '462c1cb331fdef6dbc18da2fc8253c2b774e925f'
dart = OpenDartReader(api_key)

- `finstate(corp, bsns_year, reprt_code)`
    - corp: 기업명
    - bsns_year: 사업연도
    - reprt_code: 보고서 유형 11011: 사업보고서, '11012': 반기보고서, '11013': 1분기보고서, '11014: 3분기보고서)
- 이 메서드는 아래 컬럼을 가지는 데이터프레임을 반환합니다.
    - account_nm: 계정명 ex. 자본총계, 영업이익, 당기순이익 등
    - fs_nm: 개별/연결명 ex. 연결재무제표 또는 재무제표
    - sj_nm: 재무제표명 ex. 재무상태표 또는 손익계산서
    - thstrm_dt: 당기일자  ex. 2019.12.31 현재, 2019.01.01 ~ 2019.12.31
    - thstrm_amount: 당기금액 ex. 166,009,000,000
- 계정명과 당기/전기/전전기 금액 컬럼을 바탕으로 우리가 원하는 값을 가져올 수 있습니다

In [12]:
result = dart.finstate("176440", 2021, "11011") # 2020년 래몽래인 사업보고서 내 재무 정보
result = result.loc[result.fs_nm == "연결재무제표"] # 연결재무제표 필터링

# account_nm이 당기순이익, 영업이익, 매출액 중 하나인 경우에만 가져오기
result = result.loc[result.account_nm.isin(['당기순이익', '영업이익', '매출액'])]

result = result.loc[:, ['account_nm', 'fs_nm', 'sj_nm', 'thstrm_dt', 'thstrm_amount']]
display(result)

,account_nm,fs_nm,sj_nm,thstrm_dt,thstrm_amount
9,매출액,연결재무제표,손익계산서,2021.01.01 ~ 2021.12.31,"3,761,752,259"
10,영업이익,연결재무제표,손익계산서,2021.01.01 ~ 2021.12.31,"-6,416,624,178"
12,당기순이익,연결재무제표,손익계산서,2021.01.01 ~ 2021.12.31,"-5,989,487,233"


# 1. 재무변수 수집 함수

In [13]:
def find_financial_ind(corp_nm, yr, inds):
    report = dart.finstate(corp_nm, yr) # 데이터 가져오기

    if report is None:
        # 리포트가 없으면 당기, 전기, 전전기 값 모두 제거
        data = [[corp_nm, yr] + [np.nan] * len(inds)]
        data = [[corp_nm, yr-1] + [np.nan] * len(inds)]
        data = [[corp_nm, yr-2] + [np.nan] * len(inds)]
        return pd.DataFrame(data, columns=['기업명', '연도'] + inds)
    
    else:
        report = report[report.account_nm.isin(inds)]
        if sum(report.fs_nm == '연결재무제표') > 0:
            # 연결재무제표 데이터가 있으면 연결재무제표 사용
            report = report.loc[report.fs_nm == '연결재무제표']

        else:
            # 연결재무제표 데이터가 없으면 일반재무제표 사용
            report = report.loc[report.fs_nm == '재무제표']

        data = []
        for y, c in zip([yr, yr-1, yr-2], ['thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']):
            record = [corp_nm, y]
            for ind in inds:
                # account_nm이 ind인 행의 c 컬럼 값을 가져 옴
                if sum(report.account_nm == ind) > 0:
                    value = report.loc[report.account_nm == ind, c].iloc[0]
                else:
                    value = np.nan

                record.append(value)
            
            data.append(record)

        return pd.DataFrame(data, columns=['기업명', '연도'] + inds)

In [14]:
def str_to_float(value):
    if type(value) == float:
        return value
    elif value == '-':
        return 0
    else:
        return float(value.replace(',', ''))

In [15]:
inds = ['자산총계', '부채총계', '자본총계', '매출액', '영업이익', '당기순이익', '유동자산', '유동부채', '이자비용']
display(find_financial_ind('삼성전자', 2020, inds))

,기업명,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,유동자산,유동부채,이자비용
0,삼성전자,2020,"378,235,718,000,000","102,287,702,000,000","275,948,016,000,000","236,806,988,000,000","35,993,876,000,000","26,407,832,000,000","198,215,579,000,000","75,604,351,000,000",NaN
1,삼성전자,2019,"352,564,497,000,000","89,684,076,000,000","262,880,421,000,000","230,400,881,000,000","27,768,509,000,000","21,738,865,000,000","181,385,260,000,000","63,782,764,000,000",NaN
2,삼성전자,2018,"339,357,244,000,000","91,604,067,000,000","247,753,177,000,000","243,771,415,000,000","58,886,669,000,000","44,344,857,000,000","174,697,424,000,000","69,081,510,000,000",NaN


# 2. 정상기업 재무 변수 수집

In [16]:
list_corp = pd.read_csv('./data/정상기업.csv', dtype={'종목코드':str})
list_corps = list_corp.기업명.to_list()
list_corps[:3]

['래몽래인', '툴젠', '마음AI']

In [17]:
inds = ['자산총계', '부채총계', '자본총계', '매출액', '영업이익', '당기순이익',\
    '유동부채', '유동자산', '비유동자산', '비유동부채']

list_data = pd.DataFrame()

for idx, corp_nm in enumerate(list_corps):
    print(idx+1, "/", len(list_corps))
    yr = 2022
    try:
        result = find_financial_ind(corp_nm, yr, inds)
    except:
        pass

    list_data = pd.concat([list_data, result], axis=0, ignore_index=True)
    time.sleep(0.5)

for ind in inds:
    list_data[ind] = list_data[ind].apply(str_to_float)

1 / 1455
2 / 1455
3 / 1455
4 / 1455
5 / 1455
6 / 1455
7 / 1455
8 / 1455
9 / 1455
10 / 1455
11 / 1455
12 / 1455
13 / 1455
14 / 1455
15 / 1455
16 / 1455
17 / 1455
18 / 1455
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

19 / 1455
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

20 / 1455
21 / 1455
22 / 1455
23 / 1455
24 / 1455
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

25 / 1455
26 / 1455
27 / 1455
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

28 / 1455
29 / 1455
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

30 / 1455
31 / 1455
32 / 1455
33 / 1455
34 / 1455
35 / 1455
36 / 1455
37 / 1455
38 / 1455
39 / 1455
40 / 1455
41 / 1455
42 / 1455
43 / 1455
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

44 / 1455
45 / 1455
46 / 1455
47 / 1455
48 / 1455
49 / 1455
50 / 1455
51 / 1455
52 / 1455
53 / 1455
54 / 1455
55 / 1455
56 / 1455
57 / 1455
58 / 1455
59 / 1455
60 / 1455
61 / 1455
62 / 1455
63 / 1455
64 / 1455
65 / 1455
66 / 1455
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

67 / 1455
68 

In [18]:
list_data_raw = list_data.copy()
list_data_raw.to_csv('./data/정상기업재무원본.csv', index=False)

In [19]:
list_data.sort_values(by=['기업명', '연도'], inplace=True)
list_data.drop_duplicates(inplace=True)

In [20]:
list_data.head(3)

,기업명,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,유동부채,유동자산,비유동자산,비유동부채
3236,3S,2020,4.965837e+10,1.629335e+10,3.336502e+10,2.259202e+10,4.671811e+08,-1.080358e+09,1.351997e+10,1.931293e+10,3.034544e+10,2.773384e+09
3235,3S,2021,5.461475e+10,1.806843e+10,3.654632e+10,2.347791e+10,-1.408266e+09,6.953349e+08,1.576700e+10,1.807067e+10,3.654408e+10,2.301429e+09
3234,3S,2022,6.512786e+10,2.678888e+10,3.833899e+10,2.711063e+10,1.452134e+09,1.171370e+09,2.442367e+10,2.888501e+10,3.624285e+10,2.365204e+09


In [21]:
list_data.isnull().sum()

기업명       0
연도        0
자산총계      3
부채총계      3
자본총계      9
매출액      93
영업이익      6
당기순이익    42
유동부채      3
유동자산      3
비유동자산     3
비유동부채     3
dtype: int64

# 3. 결측치 knn

In [22]:
from sklearn.impute import KNNImputer

null_cols = list_data.iloc[:, 1:].columns

imputer=KNNImputer(n_neighbors=5)
filled_list_data = imputer.fit_transform(list_data.iloc[:, 1:])
filled_list_data = pd.DataFrame(filled_list_data, columns=list_data.iloc[:, 1:].columns)

In [23]:
filled_list_data.shape, list_data.shape

((3636, 11), (3636, 12))

In [24]:
filled_list_data.isnull().sum()

연도       0
자산총계     0
부채총계     0
자본총계     0
매출액      0
영업이익     0
당기순이익    0
유동부채     0
유동자산     0
비유동자산    0
비유동부채    0
dtype: int64

In [25]:
list_data = list_data.reset_index().drop(['index'], axis=1)
list_data[null_cols] = filled_list_data

In [26]:
list_data.shape

(3636, 12)

In [27]:
list_data.isnull().sum()

기업명      0
연도       0
자산총계     0
부채총계     0
자본총계     0
매출액      0
영업이익     0
당기순이익    0
유동부채     0
유동자산     0
비유동자산    0
비유동부채    0
dtype: int64

# 4. 재무 비율 변수 구성
- https://gils-lab.tistory.com/38
- https://dacon.io/competitions/official/235946/codeshare/5805
- https://blog.naver.com/o12486vs2/222096044791

### (1) 건전성

In [28]:
# 부채비율 = 총부채 / 총자산 DR (LEV)
list_data['부채비율'] = list_data['부채총계'] / list_data['자본총계'] * 100

# 자기자본비율 DER
list_data['자기자본비율'] = list_data['부채총계'] / list_data['자산총계'] * 100

# 유동부채비율
list_data['유동부채비율'] = list_data['유동부채'] / list_data['자본총계'] * 100

# 유동부채비율
list_data['비유동부채비율'] = list_data['비유동부채'] / list_data['자본총계'] * 100


### (2) 수익성

In [29]:
# 총자산영업이익율 OI/TA
list_data['총자산영업이익율'] = list_data['영업이익'] / list_data['자산총계']

# 총자산순이익율 ROA
list_data['ROA'] = list_data['당기순이익'] / list_data['자산총계']

# 자기자본이익율 ROE
avg_eq = list_data['자본총계'].rolling(2).mean()
list_data['ROE'] = list_data['당기순이익'] / avg_eq
list_data.loc[list_data.연도 == 2020, 'ROE'] = np.nan

# 매출액영업이익율 OPM
list_data['매출액영업이익율'] = list_data['영업이익'] / list_data['매출액']

# 매출액순이익율
list_data['매출액순이익율'] = list_data['당기순이익'] / list_data['매출액']


### (3) 성장성

In [30]:
list_data['총자산증가율'] = list_data['자산총계'].diff() / list_data['자산총계'] * 100
list_data.loc[list_data.연도 == 2020, '총자산증가율'] = np.nan

list_data['매출액증가율'] = list_data['매출액'].diff() / list_data['매출액'] * 100
list_data.loc[list_data.연도 == 2020, '매출액증가율'] = np.nan

list_data['당기순이익증가율'] = list_data['당기순이익'].diff() / list_data['당기순이익'] * 100
list_data.loc[list_data.연도 == 2020, '당기순이익증가율'] = np.nan

list_data['영업이익증가율'] = list_data['영업이익'].diff() / list_data['영업이익'] * 100
list_data.loc[list_data.연도 == 2020, '영업이익증가율'] = np.nan

### (4) 유동성

In [31]:
# 유동비율 LIQ
list_data['유동비율'] = list_data['유동자산'] / list_data['유동부채']

### (5) 활동성

In [32]:
# 자산회전율
list_data['자산회전율'] = list_data['매출액'] / list_data['자산총계']

# 부채회전율
list_data['부채회전율'] = list_data['매출액'] / list_data['부채총계']

# 자본회전율
list_data['자본회전율'] = list_data['매출액'] / list_data['자본총계']


### (6) 규모

In [33]:
# 총매출액규모
list_data['총매출액규모'] = np.log(list_data['매출액'])

# 총매출액규모
list_data['총자산규모'] = np.log(list_data['자산총계'])

/Users/yeorii/opt/anaconda3/envs/yeonii/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/yeorii/opt/anaconda3/envs/yeonii/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [34]:
list_data.head()

,기업명,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,유동부채,유동자산,비유동자산,비유동부채,부채비율,자기자본비율,유동부채비율,비유동부채비율,총자산영업이익율,ROA,ROE,매출액영업이익율,매출액순이익율,총자산증가율,매출액증가율,당기순이익증가율,영업이익증가율,유동비율,자산회전율,부채회전율,자본회전율,총매출액규모,총자산규모
0,3S,2020.0,4.965837e+10,1.629335e+10,3.336502e+10,2.259202e+10,4.671811e+08,-1.080358e+09,1.351997e+10,1.931293e+10,3.034544e+10,2.773384e+09,48.833632,32.810885,40.521384,8.312249,0.009408,-0.021756,NaN,0.020679,-0.047820,NaN,NaN,NaN,NaN,1.428475,0.454949,1.386579,0.677117,23.840862,24.628433
1,3S,2021.0,5.461475e+10,1.806843e+10,3.654632e+10,2.347791e+10,-1.408266e+09,6.953349e+08,1.576700e+10,1.807067e+10,3.654408e+10,2.301429e+09,49.439795,33.083420,43.142503,6.297292,-0.025785,0.012732,0.019892,-0.059983,0.029617,9.075163,3.773305,255.372375,133.174208,1.146107,0.429882,1.299389,0.642415,23.879326,24.723570
2,3S,2022.0,6.512786e+10,2.678888e+10,3.833899e+10,2.711063e+10,1.452134e+09,1.171370e+09,2.442367e+10,2.888501e+10,3.624285e+10,2.365204e+09,69.873715,41.132741,63.704527,6.169188,0.022297,0.017986,0.031284,0.053563,0.043207,16.142265,13.399616,40.639176,196.979059,1.182665,0.416268,1.012011,0.707129,24.023192,24.899618
3,APS,2020.0,2.713477e+11,1.077549e+11,1.635928e+11,2.534595e+10,-1.087706e+10,-4.230297e+10,3.022269e+10,3.754778e+10,2.337999e+11,7.753220e+10,65.867759,39.711008,18.474345,47.393414,-0.040085,-0.155900,NaN,-0.429144,-1.669023,NaN,NaN,NaN,NaN,1.242370,0.093408,0.235219,0.154933,23.955885,26.326667
4,APS,2021.0,3.370926e+11,1.498233e+11,1.872693e+11,3.941552e+10,-3.963572e+09,2.358586e+10,6.897271e+10,1.004838e+11,2.366088e+11,8.085059e+10,80.004210,44.445744,36.830767,43.173443,-0.011758,0.069968,0.134445,-0.100559,0.598390,19.503519,35.695503,279.357351,-174.425692,1.456863,0.116928,0.263080,0.210475,24.397426,26.543623


In [35]:
list_data = list_data[list_data.연도 == 2022]
list_data.head()

,기업명,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,유동부채,유동자산,비유동자산,비유동부채,부채비율,자기자본비율,유동부채비율,비유동부채비율,총자산영업이익율,ROA,ROE,매출액영업이익율,매출액순이익율,총자산증가율,매출액증가율,당기순이익증가율,영업이익증가율,유동비율,자산회전율,부채회전율,자본회전율,총매출액규모,총자산규모
2,3S,2022.0,6.512786e+10,2.678888e+10,3.833899e+10,2.711063e+10,1.452134e+09,1.171370e+09,2.442367e+10,2.888501e+10,3.624285e+10,2.365204e+09,69.873715,41.132741,63.704527,6.169188,0.022297,0.017986,0.031284,0.053563,0.043207,16.142265,13.399616,40.639176,196.979059,1.182665,0.416268,1.012011,0.707129,24.023192,24.899618
5,APS,2022.0,3.676376e+11,1.550304e+11,2.126071e+11,4.082193e+10,-1.526963e+10,6.392560e+09,1.021253e+11,9.745597e+10,2.701816e+11,5.290513e+10,72.918737,42.169367,48.034755,24.883982,-0.041534,0.017388,0.031973,-0.374054,0.156596,8.308460,3.445238,-268.957924,74.042775,0.954278,0.111039,0.263316,0.192006,24.432485,26.630364
8,AP시스템,2022.0,5.481129e+11,2.779137e+11,2.701991e+11,4.866076e+11,9.048516e+10,8.223767e+10,2.224212e+11,3.964706e+11,1.516423e+11,5.549255e+10,102.855150,50.703741,82.317502,20.537648,0.165085,0.150038,0.356206,0.185951,0.169002,21.782520,-8.658585,30.486565,28.932548,1.782522,0.887787,1.750930,1.800922,26.910724,27.029747
11,AP위성,2022.0,1.092994e+11,2.394835e+10,8.535104e+10,5.360413e+10,4.494271e+09,4.077471e+09,2.380234e+10,9.807139e+10,1.122799e+10,1.460048e+08,28.058646,21.910778,27.887582,0.171064,0.041119,0.037306,0.048992,0.083842,0.076066,-1.728078,25.002563,65.622301,81.260622,4.120241,0.490434,2.238323,0.628043,24.704892,25.417357
14,BGF에코머티리얼즈,2022.0,3.521437e+11,1.501987e+11,2.019451e+11,2.633016e+11,1.592546e+10,2.887958e+10,1.177644e+11,2.531803e+11,9.896346e+10,3.243424e+10,74.375997,42.652658,58.315074,16.060923,0.045224,0.082011,0.161303,0.060484,0.109683,12.886848,22.729716,97.253119,-39.828060,2.149888,0.747711,1.753022,1.303828,26.296566,26.587305


In [36]:
list_data = list_data.reset_index().drop(['index'], axis=1)
list_data.shape

(1212, 31)

In [37]:
list_data.isnull().sum()

기업명         0
연도          0
자산총계        0
부채총계        0
자본총계        0
매출액         0
영업이익        0
당기순이익       0
유동부채        0
유동자산        0
비유동자산       0
비유동부채       0
부채비율        0
자기자본비율      0
유동부채비율      0
비유동부채비율     0
총자산영업이익율    0
ROA         0
ROE         0
매출액영업이익율    0
매출액순이익율     0
총자산증가율      0
매출액증가율      4
당기순이익증가율    0
영업이익증가율     0
유동비율        1
자산회전율       0
부채회전율       0
자본회전율       0
총매출액규모      0
총자산규모       0
dtype: int64

In [40]:
list_data.head()

,기업명,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,유동부채,유동자산,비유동자산,비유동부채,부채비율,자기자본비율,유동부채비율,비유동부채비율,총자산영업이익율,ROA,ROE,매출액영업이익율,매출액순이익율,총자산증가율,매출액증가율,당기순이익증가율,영업이익증가율,유동비율,자산회전율,부채회전율,자본회전율,총매출액규모,총자산규모
0,3S,2022.0,6.512786e+10,2.678888e+10,3.833899e+10,2.711063e+10,1.452134e+09,1.171370e+09,2.442367e+10,2.888501e+10,3.624285e+10,2.365204e+09,69.873715,41.132741,63.704527,6.169188,0.022297,0.017986,0.031284,0.053563,0.043207,16.142265,13.399616,40.639176,196.979059,1.182665,0.416268,1.012011,0.707129,24.023192,24.899618
1,APS,2022.0,3.676376e+11,1.550304e+11,2.126071e+11,4.082193e+10,-1.526963e+10,6.392560e+09,1.021253e+11,9.745597e+10,2.701816e+11,5.290513e+10,72.918737,42.169367,48.034755,24.883982,-0.041534,0.017388,0.031973,-0.374054,0.156596,8.308460,3.445238,-268.957924,74.042775,0.954278,0.111039,0.263316,0.192006,24.432485,26.630364
2,AP시스템,2022.0,5.481129e+11,2.779137e+11,2.701991e+11,4.866076e+11,9.048516e+10,8.223767e+10,2.224212e+11,3.964706e+11,1.516423e+11,5.549255e+10,102.855150,50.703741,82.317502,20.537648,0.165085,0.150038,0.356206,0.185951,0.169002,21.782520,-8.658585,30.486565,28.932548,1.782522,0.887787,1.750930,1.800922,26.910724,27.029747
3,AP위성,2022.0,1.092994e+11,2.394835e+10,8.535104e+10,5.360413e+10,4.494271e+09,4.077471e+09,2.380234e+10,9.807139e+10,1.122799e+10,1.460048e+08,28.058646,21.910778,27.887582,0.171064,0.041119,0.037306,0.048992,0.083842,0.076066,-1.728078,25.002563,65.622301,81.260622,4.120241,0.490434,2.238323,0.628043,24.704892,25.417357
4,BGF에코머티리얼즈,2022.0,3.521437e+11,1.501987e+11,2.019451e+11,2.633016e+11,1.592546e+10,2.887958e+10,1.177644e+11,2.531803e+11,9.896346e+10,3.243424e+10,74.375997,42.652658,58.315074,16.060923,0.045224,0.082011,0.161303,0.060484,0.109683,12.886848,22.729716,97.253119,-39.828060,2.149888,0.747711,1.753022,1.303828,26.296566,26.587305


In [41]:
list_data.to_csv('./data/정상기업재무.csv', index=False)

# 실패한 애들 알아보기

In [42]:
list_corp = pd.read_csv('./data/정상기업.csv', dtype={'종목코드':str})
print(list_corp.shape)
list_corp.head()

(1455, 4)


,기업명,종목코드,상장일,지속기간
0,래몽래인,200350,2021-12-30,670 days
1,툴젠,199800,2021-12-10,690 days
2,마음AI,377480,2021-11-23,707 days
3,알비더블유,361570,2021-11-22,708 days
4,바이옵트로,222160,2021-11-18,712 days


In [43]:
list_done = pd.read_csv('./data/정상기업재무.csv', dtype={'종목코드':str})
print(list_done.shape)
list_done.head()

(1212, 31)


,기업명,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,유동부채,유동자산,비유동자산,비유동부채,부채비율,자기자본비율,유동부채비율,비유동부채비율,총자산영업이익율,ROA,ROE,매출액영업이익율,매출액순이익율,총자산증가율,매출액증가율,당기순이익증가율,영업이익증가율,유동비율,자산회전율,부채회전율,자본회전율,총매출액규모,총자산규모
0,3S,2022.0,6.512786e+10,2.678888e+10,3.833899e+10,2.711063e+10,1.452134e+09,1.171370e+09,2.442367e+10,2.888501e+10,3.624285e+10,2.365204e+09,69.873715,41.132741,63.704527,6.169188,0.022297,0.017986,0.031284,0.053563,0.043207,16.142265,13.399616,40.639176,196.979059,1.182665,0.416268,1.012011,0.707129,24.023192,24.899618
1,APS,2022.0,3.676376e+11,1.550304e+11,2.126071e+11,4.082193e+10,-1.526963e+10,6.392560e+09,1.021253e+11,9.745597e+10,2.701816e+11,5.290513e+10,72.918737,42.169367,48.034755,24.883982,-0.041534,0.017388,0.031973,-0.374054,0.156596,8.308460,3.445238,-268.957924,74.042775,0.954278,0.111039,0.263316,0.192006,24.432485,26.630364
2,AP시스템,2022.0,5.481129e+11,2.779137e+11,2.701991e+11,4.866076e+11,9.048516e+10,8.223767e+10,2.224212e+11,3.964706e+11,1.516423e+11,5.549255e+10,102.855150,50.703741,82.317502,20.537648,0.165085,0.150038,0.356206,0.185951,0.169002,21.782520,-8.658585,30.486565,28.932548,1.782522,0.887787,1.750930,1.800922,26.910724,27.029747
3,AP위성,2022.0,1.092994e+11,2.394835e+10,8.535104e+10,5.360413e+10,4.494271e+09,4.077471e+09,2.380234e+10,9.807139e+10,1.122799e+10,1.460048e+08,28.058646,21.910778,27.887582,0.171064,0.041119,0.037306,0.048992,0.083842,0.076066,-1.728078,25.002563,65.622301,81.260622,4.120241,0.490434,2.238323,0.628043,24.704892,25.417357
4,BGF에코머티리얼즈,2022.0,3.521437e+11,1.501987e+11,2.019451e+11,2.633016e+11,1.592546e+10,2.887958e+10,1.177644e+11,2.531803e+11,9.896346e+10,3.243424e+10,74.375997,42.652658,58.315074,16.060923,0.045224,0.082011,0.161303,0.060484,0.109683,12.886848,22.729716,97.253119,-39.828060,2.149888,0.747711,1.753022,1.303828,26.296566,26.587305


In [44]:
list_corps_done = list_done.기업명

list_corps_fail = list_corp[~list_corp.기업명.isin(list_corps_done)]
print(list_corps_fail.shape)
list_corps_fail.head()

(243, 4)


,기업명,종목코드,상장일,지속기간
17,지아이텍,382480,2021-10-21,740 days
18,코닉오토메이션,391710,2021-10-15,746 days
23,에스앤디,260970,2021-09-29,762 days
26,와이엠텍,273640,2021-09-10,781 days
28,라이콤,388790,2021-09-03,788 days


In [45]:
list_corps_fail.to_csv('./data/정상기업재무실패.csv', index=False)